In [1]:
import pandas as pd

df = pd.read_csv('patient_data.csv')

In [2]:
# from ydata_profiling import ProfileReport

# profile = ProfileReport(df, title='Profiling Report', minimal=True)
# profile.to_file("data_profile.html")

In [3]:
from preprocessing import preprocess_data

df = pd.read_csv('patient_data.csv')
df = preprocess_data(df)

In [4]:
df.head()

,female,age,BinDaysfromHostoICUAdmit,hosplos,iculos,apache2,ADMITLOCATIONTXT,PrimaryDiagnosis,Overflow,Mechanical.Ventilation,vasoactive,Oncatheter,OnInsulin,readmit72
0,0.0,42.0,2.0,2259780.0,763920.0,23.0,4.0,262.0,0.0,1.0,1.0,0.0,1.0,False
1,1.0,61.0,1.0,4215360.0,649800.0,14.0,3.0,136.0,1.0,1.0,1.0,0.0,1.0,True
4,0.0,68.0,2.0,1995120.0,159780.0,13.0,0.0,259.0,1.0,1.0,0.0,1.0,0.0,False
6,0.0,74.0,1.0,5871420.0,554700.0,20.0,0.0,116.0,0.0,1.0,0.0,1.0,1.0,False
7,0.0,61.0,1.0,47679300.0,903660.0,21.0,0.0,264.0,1.0,1.0,1.0,0.0,0.0,False


In [5]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='readmit72')
y = df['readmit72']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from torch.utils.data import DataLoader, TensorDataset
import torch

# Convert DataFrame to PyTorch Tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor.view(-1, 1))
test_dataset = TensorDataset(X_test_tensor, y_test_tensor.view(-1, 1))

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=6, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=6, shuffle=False)

In [7]:
from MLP import MLP
import pytorch_lightning as pl

model = MLP()
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_dataloader)
trainer.test(model, test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 896   
1 | fc2  | Linear | 2.1 K 
2 | fc3  | Linear | 33    
--------------------------------
3.0 K     Trainable params
0         Non-trainable params
3.0 K     Total params
0.012     Total estimated model params size (MB)
c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.955367922782898     │
│         test_loss         │    0.6931466460227966     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6931466460227966, 'test_acc': 0.955367922782898}]